Trying fastaudio for myself : https://www.kaggle.com/scart97/fastaudio-starter-kit

In [ ]:
!pip install --upgrade light-the-torch
!ltt install torch torchvision torchaudio
!pip install --upgrade git+http://github.com/fastaudio/fastaudio.git

In [ ]:
# this is needed for the library
import pkg_resources
def placeholder(x):
    raise pkg_resources.DistributionNotFound
pkg_resources.get_distribution = placeholder

In [ ]:
import pandas as pd
from fastaudio.all import *
from fastai.vision.all import *

In [ ]:
path = Path("../input/rfcx-species-audio-detection")
for i in path.ls():
    print(i)

In [ ]:
train_path = path / 'train'
test_path = path / 'test'

In [ ]:
train_files = get_audio_files(train_path)
len(train_files)

In [ ]:
index =math.floor(random.random()*4727)
print(index)

# random bird chirping from amazon!
audio = AudioTensor.create(train_files[index])
audio.show()

In [ ]:
df_train_tp = pd.read_csv(path/ 'train_tp.csv')
df_train_tp["recording_id"] = df_train_tp["recording_id"].map(lambda x: "train/"+x)
df_train_tp.head()

In [ ]:
df_train_tp.iloc[:]

In [ ]:
df_train_tp = df_train_tp.drop(['t_min', 't_max', 'f_min', 'f_max', 'songtype_id'],axis=1)
df_train_tp['species_id'] = df_train_tp['species_id'].astype(str)
df_train_tp.head()

In [ ]:
df_train_tp['species_id'] = df_train_tp.groupby('recording_id')['species_id'].transform(",".join)
df_train_tp = df_train_tp.reset_index()

In [ ]:
df_train_tp

In [ ]:
# this part is wizadry
audio_to_spec = AudioToSpec.from_cfg(AudioConfig.BasicMelSpectrogram(n_fft=512))
data_augmentation = [AddNoise(color=NoiseColor.White, noise_level=0.1), SignalShifter(max_pct=0.3)]

blocks = DataBlock(blocks=(AudioBlock, MultiCategoryBlock), get_x = ColReader('recording_id', pref=str(path.resolve()) + "/" , suff='.flac'),
                   get_y = ColReader('species_id', label_delim=','),
                   item_tfms = data_augmentation,
                   batch_tfms = audio_to_spec,
                   splitter = RandomSplitter(valid_pct=0.2, seed=42))

In [ ]:
dls = blocks.dataloaders(df_train_tp,bs=24)

In [ ]:
dls.show_batch(ncols=3, nrows=2, figsize=(20, 10))

In [ ]:
learner = cnn_learner(dls, resnet18, config={"n_in":1})

In [ ]:
learner.lr_find()

In [ ]:
learner.fine_tune(10, base_lr=5e-2)

In [ ]:
learner.recorder.plot_loss()

In [ ]:
submission_df = pd.read_csv(path / 'sample_submission.csv')
submission_df["recording_id"] = submission_df["recording_id"].map(lambda x: "test/"+x)
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)

It works !